In [ ]:
import requests
import pandas as pd
import os
import datetime
import concurrent.futures
import re
import json
import threading
from rich.progress import Progress, BarColumn, TextColumn, TimeElapsedColumn, TimeRemainingColumn
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

PATH = "C:\\Users\\Haroldo Duraes\\Desktop\\GOvGO\\v0\\#DATA\\PNCP\\"
IN_PATH = PATH + "CONTRATAÇÕES\\"
OUT_PATH = PATH + "ITENS\\"
IN_FILE = "CONTRATAÇÕES_PNCP_01_2025.xlsx"
OUT_FILE = "ITENS_CONTRATAÇÕES_PNCP_01_2025.xlsx"

# Lista de campos dos itens conforme o exemplo do arquivo itens.json (fixa)
ITEM_FIELDS = [
    "numeroItem",
    "descricao",
    "materialOuServico",
    "materialOuServicoNome",
    "valorUnitarioEstimado",
    "valorTotal",
    "quantidade",
    "unidadeMedida",
    "orcamentoSigiloso",
    "itemCategoriaId",
    "itemCategoriaNome",
    "patrimonio",
    "codigoRegistroImobiliario",
    "criterioJulgamentoId",
    "criterioJulgamentoNome",
    "situacaoCompraItem",
    "situacaoCompraItemNome",
    "tipoBeneficio",
    "tipoBeneficioNome",
    "incentivoProdutivoBasico",
    "dataInclusao",
    "dataAtualizacao",
    "temResultado",
    "imagem",
    "aplicabilidadeMargemPreferenciaNormal",
    "aplicabilidadeMargemPreferenciaAdicional",
    "percentualMargemPreferenciaNormal",
    "percentualMargemPreferenciaAdicional",
    "ncmNbsCodigo",
    "ncmNbsDescricao",
    "catalogo",
    "categoriaItemCatalogo",
    "catalogoCodigoItem",
    "informacaoComplementar"
]

def remove_illegal_chars(value):
    if isinstance(value, str):
        return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', value)
    return value

def clean_dataframe(df):
    return df.apply(lambda col: col.map(remove_illegal_chars))

# Processa uma linha usando apenas o campo "numeroControlePNCP".
def process_row(row):
    numeroControle = str(row.get("numeroControlePNCP", "")).strip()
    if not numeroControle:
        return []
    try:
        # Tentar reconhecer o formato pelo padrão esperado
        if not re.match(r'^\d+-\d+-\d+/\d+$', numeroControle):
            # Formato inválido, pular silenciosamente
            return []
            
        # Resto do código permanece o mesmo...
        parts = numeroControle.split("-")
        if len(parts) != 3:
            return []
        cnpj = parts[0]
        # Ignora o prefixo (parts[1])
        seq_and_year = parts[2].split("/")
        if len(seq_and_year) != 2:
            return []
        seq = seq_and_year[0]
        anoCompra = seq_and_year[1]
        # Remove zeros à esquerda do sequencial
        sequencialCompra = str(int(seq))
        # Monta a URL:
        url = f"https://pncp.gov.br/api/pncp//v1/orgaos/{cnpj}/compras/{anoCompra}/{sequencialCompra}/itens"
        # Definindo timeout para evitar travamentos
        response = requests.get(url, timeout=20)
        if response.status_code != 200:
            return []
        itens = response.json()  # espera-se que seja uma lista
        resultados = []
        for item in itens:
            registro = {"numeroControlePNCP": numeroControle}
            for campo in ITEM_FIELDS:
                registro[campo] = item.get(campo)
            resultados.append(registro)
        return resultados
    except requests.exceptions.Timeout:
        return []
    except Exception as e:
        return [{"numeroControlePNCP": numeroControle, "erro": str(e)}]

# Processa uma aba (sheet) do IN_FILE.
def process_sheet(sheet_name, df, progress):
    # Remove linhas totalmente vazias
    df = df.dropna(how="all")
    progress.console.log(f"Sheet '{sheet_name}' lida com {len(df)} linhas.")
    resultados = []
    total_rows = len(df)
    task_id = progress.add_task(f"[bold yellow]  {sheet_name} (contratações)", total=total_rows)

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {executor.submit(process_row, row): idx for idx, row in df.iterrows()}
        for future in concurrent.futures.as_completed(future_to_index):
            try:
                linhas = future.result()
                for linha in linhas:
                    if 'erro' in linha:
                        progress.console.log(f"[red]Erro linha '{linha['numeroControlePNCP']}': {linha['erro']}")
                    else:
                        resultados.append(linha)
            except Exception as e:
                progress.console.log(f"[red]Erro inesperado na execução da thread: {str(e)}")
            progress.update(task_id, advance=1)

    progress.remove_task(task_id)

    colunas_saida = ["numeroControlePNCP"] + ITEM_FIELDS
    if resultados:
        sheet_df = pd.DataFrame(resultados)
        sheet_df = clean_dataframe(sheet_df)
        sheet_df = sheet_df.reindex(columns=colunas_saida)
    else:
        sheet_df = pd.DataFrame(columns=colunas_saida)

    return sheet_df

# Variáveis globais para escrita imediata
workbook_lock = threading.Lock()
# Cria um novo workbook e remove a folha padrão.
wb = Workbook()
wb.remove(wb.active)

def write_sheet(sheet_name, df, output_file, progress):
    with workbook_lock:
        # Se a aba já existir, removê-la.
        if sheet_name in wb.sheetnames:
            ws = wb[sheet_name]
            wb.remove(ws)
        ws = wb.create_sheet(title=sheet_name)
        from openpyxl.utils.dataframe import dataframe_to_rows
        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(r)
        wb.save(output_file)
        progress.console.log(f"Sheet '{sheet_name}' gravada com {len(df)} linhas.")




In [ ]:
def debug_sheet(sheet_name):
    """Função para testar apenas uma aba específica"""
    xls = pd.ExcelFile(IN_PATH + IN_FILE)
    
    if sheet_name not in xls.sheet_names:
        print(f"Aba '{sheet_name}' não encontrada no arquivo!")
        return
    
    df_sheet = pd.read_excel(xls, sheet_name=sheet_name)
    print(f"Aba '{sheet_name}' carregada com {len(df_sheet)} linhas.")
    
    # Verifique e mostre os primeiros registros para analisar
    print("\nPrimeiros 5 registros:")
    print(df_sheet.head())
    
    # Verificar números de controle
    print("\nVerificando números de controle...")
    invalid_controls = []
    for idx, row in df_sheet.iterrows():
        num_controle = str(row.get("numeroControlePNCP", "")).strip()
        if num_controle and not re.match(r'^\d+-\d+-\d+/\d+$', num_controle):
            invalid_controls.append((idx, num_controle))
    
    if invalid_controls:
        print(f"Encontrados {len(invalid_controls)} números de controle com formato inválido:")
        for idx, ctrl in invalid_controls[:10]:
            print(f"  Linha {idx}: '{ctrl}'")
    else:
        print("Todos os números de controle têm formato válido.")
    
    # Adicione mais verificações conforme necessário...

In [ ]:
def main():
    # Lê o arquivo Excel com todas as abas
    xls = pd.ExcelFile(IN_PATH + IN_FILE)
    sheets = xls.sheet_names  # Lista de abas
    
    # Cria uma instância global de Progress (Rich) com colunas customizadas
    progress = Progress(
        TextColumn("[bold yellow]{task.description}"),
        BarColumn(complete_style="green", finished_style="bright_green"),
        "[progress.percentage]{task.percentage:>3.0f}%",
        TimeElapsedColumn(),
        TimeRemainingColumn(),
    )
    
    with progress:
        # Task externa: Batch de abas
        outer_task = progress.add_task("[bold green]Batch mensal", total=len(sheets))
        with concurrent.futures.ThreadPoolExecutor(max_workers=13) as executor:
            future_to_sheet = {}
            for sheet in sheets:
                df_sheet = pd.read_excel(xls, sheet_name=sheet)
                future = executor.submit(process_sheet, sheet, df_sheet, progress)
                future_to_sheet[future] = sheet
            for future in concurrent.futures.as_completed(future_to_sheet):
                sheet = future_to_sheet[future]
                try:
                    sheet_df = future.result()
                    # Escreve imediatamente esta aba no arquivo de saída
                    write_sheet(sheet, sheet_df, OUT_PATH + OUT_FILE, progress)
                except Exception as e:
                    # Salvando o dataframe problemático para análise
                    try:
                        progress.console.log(f"[red]Erro ao processar a aba '{sheet}': {str(e)}")
                        
                        # Salvar o dataframe da aba com problema para análise
                        df_problematico = pd.read_excel(xls, sheet_name=sheet)
                        debug_file = OUT_PATH + f"DEBUG_{sheet}_data.xlsx"
                        df_problematico.to_excel(debug_file, index=False)
                        progress.console.log(f"[yellow]DataFrame da aba '{sheet}' salvo em {debug_file} para análise")
                        
                        # Verificar se há números de controle com formato inválido
                        invalid_controls = []
                        for idx, row in df_problematico.iterrows():
                            num_controle = str(row.get("numeroControlePNCP", "")).strip()
                            if num_controle and not re.match(r'^\d+-\d+-\d+/\d+$', num_controle):
                                invalid_controls.append((idx, num_controle))
                        
                        if invalid_controls:
                            progress.console.log(f"[yellow]Encontrados {len(invalid_controls)} números de controle com formato inválido:")
                            for idx, ctrl in invalid_controls[:5]:  # mostrar apenas os primeiros 5
                                progress.console.log(f"  Linha {idx}: '{ctrl}'")
                            if len(invalid_controls) > 5:
                                progress.console.log(f"  ... (mais {len(invalid_controls)-5} registros)")
                    except Exception as inner_e:
                        progress.console.log(f"[red]Erro ao tentar diagnosticar problema: {str(inner_e)}")
                finally:
                    progress.update(outer_task, advance=1)
        progress.remove_task(outer_task)
    
    print("\nPlanilha de itens consolidada salva em:", OUT_FILE)

In [8]:

if __name__ == "__main__":
    main()

Output()

[19:23:27] Sheet '2025-03' lida com 35185 linhas.                                                 1982207204.py:112

[19:52:27] Erro ao processar a aba '2025-03': Cannot convert {'id': 1, 'nome': 'Catálogo do         566466200.py:33
           Compras.gov.br', 'descricao': 'Catálogo de bens e serviços do Compras.gov.br',                          
           'dataInclusao': '2021-12-22T20:10:19', 'dataAtualizacao': '2024-11-13T21:21:13',                        
           'statusAtivo': True, 'url': 'https://catalogo.compras.gov.br/cnbs-web/busca'} to Excel                  

[19:52:54] DataFrame da aba '2025-03' salvo em C:\Users\Haroldo                                     566466200.py:39
           Duraes\Desktop\GOvGO\v0\#DATA\PNCP\ITENS\DEBUG_2025-03_data.xlsx para análise                           


Planilha de itens consolidada salva em: ITENS_CONTRATAÇÕES_PNCP_01_2025.xlsx


In [9]:
df

NameError: name 'df' is not defined